## Cleaning and Retrieval of Data
My hypothesis requires me to collect data on the different choice of educations as well as their different entrance requirements. Most of this information is already contained in the dataset found on """, but it's missing the education's admission quotient. Each education (entry) is listed with a url to the website where this piece of information can be found through webscraping. Separate datasets for admission quotients do exist, but with no consistency in how the choice of educations are named, and therefore impossible to do a straight-forward .merge/join or VLOOKUP - webscraping rather than carefully aligning the format and becomes the simpler option (and probably less prone to errors).

The data on future earnings is from surveys collected intermittently. Most survey data, hwever, is based on incredibly small sample sizes (most of which n < 20), not to mention a great share of them don't have any data on it at all. So, this limited dataset of ~300 choices of education is further narrowed down based on the availability of the data, and all numbers, even assuming that it's from a representative sample, is associated with low levels of confidence. Any conclusions drawn on this data have to be done so carefully and will inherently be limited.

#### Modules used
* requests for retrieving html pages associated with each url
* bs4 (BeautifulSoup) for searching through the html page and getting the admission quotient slightly easier
* pandas for retrieving data from excel file into a manageable dataframe allowing for easier manipulation of the data
* sleep() from time module to avoid sending html requests in too rapid a succession

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

#### About the data
As aforementioned, most of the data is contained in the dataset "ufm_samlet_23jun2022" except for the admission quotients. While it's necessary to look at the bachelor educations for the admission quotient, only the "kandidatuddannelse" of the same name has information regarding salary expectations. Proceeding to the "kandidatuddannelse" (basically a Master's) is usually guaranteed following admission in the bachelor's of the same name. The implications of this are, that initially, two dataframes are created based on the spreadsheet: one for the bachelors from which the admission quotients are retrieved; another for the "Kandidatuddannelse" which will contain the salary and job prospects. Only at the end will they be joined where the choice of education is the same - that is, we assume the choice of "kandidatuddannelse" will directly follow from the choice of bachelor's of same name. (Note, this mapping is not always possible, and therefore some entries will be lost in the joining of the dataframes) 

In [2]:
df = pd.read_excel("ufm_samlet_23jun2022.xlsx")

# the columns used from the spreadsheet
df = df[["Titel", "url", "displaydocclass", "hovedinsttx",\
         "ledighed_nyudd", "maanedloen_nyudd", "ledighed_10aar", "maanedloen_10aar"]]

# df for "kandidatuddannelse" and job and salary prospects. Also, institution and title of education choice for identifier
# Clearing entries where "Uddannelsen på landsplan" as they're just an agglomeration of the other entries when you can take
# the same education at multiple institutions
df_kandidat = df.loc[(df["displaydocclass"] == "Kandidatuddannelse") & (df["hovedinsttx"] != "Uddannelsen på landsplan")]
df_kandidat = df_kandidat[["Titel", "hovedinsttx", "ledighed_nyudd", "maanedloen_nyudd", "ledighed_10aar", "maanedloen_10aar"]]

# df for bachelor's. Only getting url for admission quotients and title and institution for unique identifier
df_bachelor = df.loc[(df["displaydocclass"] == "Bacheloruddannelse") & (df["hovedinsttx"] != "Uddannelsen på landsplan")]
df_bachelor = df_bachelor[["Titel", "url", "hovedinsttx"]]

df_kandidat.reset_index(inplace=True)
df_bachelor.reset_index(inplace=True)

#### Webscraping

The goal here is to get an array of the admission quotients, such that it can easily be added as a column later. This requires 1:1 overlap with the entries, and through trial and error I found out storing the titles was useful to ensure no duplicates in the list. 

The admission quotient is always stored in the same format inside the div with the class "views-row views-row-6", so too creative an endeavour, this webscraping was not.

A few exceptions do emerge and they're simply stored for manual inspection. Upon looking at the 4 problematic entries, it becomes clear that it happens, because the dataset is not fully up-to-date but is a year old: these problematic entries are remnants of a lost time, when one could portugese-and-brasilian studies. They're just deleted later.

In [3]:
# Getting quotients from bachelor admission
quotients = []
# storing webscraped education choices to ensure that the same is not cited multiple times
titles_done = []

# iterating through each url (entry)
# iterating through df painfully bad, but.... it was easier
for index, row in df_bachelor.iterrows():
    quotient_temp = []

    fixed_url = "https://" + row["url"]
    r = requests.get(fixed_url)
    try:
        soup = BeautifulSoup(r.text, "html.parser")
        div = soup.find("div", class_="views-row views-row-6")
        table = div.find("table")

        tbody = table.find("tbody")
        trs = tbody.find_all("tr")
        
        
        for tr in trs:
            if tr.find("b"):
                continue
            else:
                tds = tr.find_all("td")              
                sub_counter = 0
                for td in tds:
                    # pattern of every third,
                    # 0 locatoin
                    # 1 average required to get in immediately
                    # 2 average for "standby quotient"
                    if sub_counter % 3 == 1:
                        quotient_temp.append(td.text)

                    sub_counter += 1
                    
        if row["Titel"] not in titles_done:
            quotients.append(quotient_temp)
            titles_done.append(row["Titel"])
            
        sleep(0.2) # -> will take about 300 * 0.2 seconds = 1 minute
        
        
    # # exception if choice of study no longer exists. It will spit out the index number
    # and the data associated with it, such that manual inspection is possible
    except AttributeError: 
        print("Problem with", fixed_url)
        print("Whose index is,", index)
        print(row)
        print("\n")
    
print("Done executing")
        # problems with indexes
        # 141, 161, 222, 327

Problem with https://www.ug.dk/uddannelser/bachelorogkandidatuddannelser/bacheloruddannelser/humanistiskebacheloruddannelser/sprog/portugisiske-og-brasilianske-studier
Whose index is, 141
index                                                       1518
Titel                       Portugisiske og brasilianske studier
url            www.ug.dk/uddannelser/bachelorogkandidatuddann...
hovedinsttx                               Københavns Universitet
Name: 141, dtype: object


Problem with https://www.ug.dk/uddannelser/bachelorogkandidatuddannelser/bacheloruddannelser/naturvidenskabeligebacheloruddannelser/biologigeografigeologimv/biokemi-og-molekylaer-biologi
Whose index is, 161
index                                                       1549
Titel                               Biokemi og molekylær biologi
url            www.ug.dk/uddannelser/bachelorogkandidatuddann...
hovedinsttx                                 Syddansk Universitet
Name: 161, dtype: object


Problem with https://www.ug.dk/

#### Removing old data
After running, issues arise with indexes 141, 161, 222, 327. They are dropped from the dataframe.

In [4]:
# dropping problematic indexes -> choice of study no longer exists [verified manually]
problematic_indexes = [141, 161, 222, 327]
for i in problematic_indexes:
    df_bachelor.drop(i, inplace=True)

#### Adding Admission Quotients
There's still one error with the array of admission quotients: it has a length of 325, whereas the dataframe has a length of 324. For some reason, "Kun kvote 2" snuck in. Beyond that, there's also the question of how to deal with "AO". Most universities have enough places for everyone who tried to apply got in, so there was effectively no admission quotient. That's translated to become a 0 in the dataset, but I'm still not sure there weren't better solutions for that, as it looks weird on data visualisations.

In [5]:
quotient_column = [j for i in quotients for j in i if j != "Kun kvote 2"]
quotient_column = [float(x.replace(",", ".")) if x != "AO" else 0 for x in quotient_column]
df_bachelor = df_bachelor.assign(Adgangskvotienter=quotient_column)

#### Merging the Two Dataframes
To address the lack of a common unique identifier that could be used to join the two dataframes without error, I opted to create a new one. This one combines the title of the choice of education and the institution in a common format across the dataframes.

After that, the two databases can be joined on that column. Some columns are dropped as they are superfluous. The same goes for duplicate entries that have persisted from the original dataset used. Finally, the dataframe is converted into a CSV file so it's ready for R and Tableau.

In [6]:
df_kandidat["Uddannelse"] = df_kandidat["Titel"] + " - " + df_kandidat["hovedinsttx"]
df_bachelor["Uddannelse"] = df_bachelor["Titel"] + " - " + df_bachelor["hovedinsttx"]
df2 = df_bachelor.merge(df_kandidat, left_on="Uddannelse", right_on="Uddannelse")
df2.drop(columns=["Titel_x", "hovedinsttx_x", "index_x", "index_y", "Titel_y", "hovedinsttx_y"], inplace=True)
df2.reset_index(inplace=True)
df2 = df2.drop_duplicates(subset=["Uddannelse", "url"])
df2.to_csv("education_earnings_data.csv", index=False)